In [1]:
import os
import pandas as pd
from IPython.display import IFrame
import folium
import seaborn as sns
import glob
import networkx as nx
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# reading and formatting the data

In [2]:
loc='/Users/wentingjiang/Desktop/mda_kul_local/'

In [11]:
def read_noise_data(folder_path='export_40/'):
   # folder_path = 'export_40/'  # Replace with the path to your folder
    all_files = glob.glob(folder_path + "/*.csv")

    dfs = []
    for filename in all_files:
        data = pd.read_csv(filename, index_col=None, header=0, sep=';')

        data.loc[data['description'] == 'MP08bis - Vrijthof', 'description'] = 'MP 08: bis - Vrijthof'
        #data.loc[data['description'] == 'Naamsestraat 35  Maxim', 'description'] = 'MP 01: Naamsestraat 35 Maxim'

       #location_name = filename.split('.')[0].split('/')[-1].split('_')[-1]
       # print(location_name)
       # data['location']=location_name
        dfs.append(data)

    df_percentiled = pd.concat(dfs, axis=0, ignore_index=True)
    return df_percentiled

def create_location_name_dict(df_percentiled):
    pair_dict = {p.split(': ')[0]: p.split(': ')[1] for p in df_percentiled['description']}
    return pair_dict

def truncate_time_to_min(df):
    df['timestamp'] = df['result_timestamp'].str[:-7]
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M')
    
    return df.drop('result_timestamp', axis=1)

def print_na_percent(df):
    print(df.isna().sum()/df.shape[0])

def remove_noise_undetected(df):
    #print() how many removed
    return df[~df['noise_event_laeq_primary_detected_certainty'].isna()]

## Noise percentiled

In [5]:
# this data was not used for now
df_percentiled= read_noise_data(loc+'export_40/')
df_percentiled.head()

,#object_id,description,result_timestamp,laf005_per_hour,laf005_per_hour_unit,laf01_per_hour,laf01_per_hour_unit,laf05_per_hour,laf05_per_hour_unit,laf10_per_hour,...,laf90_per_hour,laf90_per_hour_unit,laf95_per_hour,laf95_per_hour_unit,laf98_per_hour,laf98_per_hour_unit,laf99_per_hour,laf99_per_hour_unit,laf995_per_hour,laf995_per_hour_unit
0,255439,MP 01: Naamsestraat 35 Maxim,07/03/2022 13:00:00.000,65.4,dB(A),64.0,dB(A),60.5,dB(A),58.0,...,47.4,dB(A),46.4,dB(A),45.2,dB(A),44.8,dB(A),44.0,dB(A)
1,255439,MP 01: Naamsestraat 35 Maxim,07/03/2022 14:00:00.000,72.2,dB(A),70.8,dB(A),65.8,dB(A),62.6,...,45.2,dB(A),43.9,dB(A),42.4,dB(A),41.4,dB(A),40.4,dB(A)
2,255439,MP 01: Naamsestraat 35 Maxim,07/03/2022 15:00:00.000,74.5,dB(A),72.2,dB(A),66.0,dB(A),63.5,...,48.6,dB(A),47.3,dB(A),45.8,dB(A),45.2,dB(A),44.5,dB(A)
3,255439,MP 01: Naamsestraat 35 Maxim,07/03/2022 16:00:00.000,72.0,dB(A),70.3,dB(A),65.0,dB(A),62.0,...,45.4,dB(A),43.8,dB(A),42.4,dB(A),41.5,dB(A),40.9,dB(A)
4,255439,MP 01: Naamsestraat 35 Maxim,07/03/2022 17:00:00.000,72.6,dB(A),71.2,dB(A),66.6,dB(A),63.6,...,45.5,dB(A),43.8,dB(A),42.3,dB(A),41.4,dB(A),40.7,dB(A)


In [7]:
location_dict=create_location_name_dict(df_percentiled)
location_dict

{'MP 01': 'Naamsestraat 35  Maxim',
 'MP 03': 'Naamsestraat 62 Taste',
 'MP 05': 'Calvariekapel KU Leuven',
 'MP 06': 'Parkstraat 2 La Filosovia',
 'MP 08': 'bis - Vrijthof',
 'MP 02': 'Naamsestraat 57 Xior',
 'MP 07': 'Naamsestraat 81',
 'MP 04': 'His & Hears'}

## Noise level

In [11]:
loc+'export_42/**/'

'/Users/wentingjiang/Desktop/mda_kul_local/export_42/**/'

In [ ]:
# limiting to the month below will give 6 locations with noise events

df_noise_level=pd.DataFrame()
months = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Oct', 'Nov', 'Dec']
for month in months:
    temp = read_noise_data(loc+'export_42/'+month+'/')
    df_noise_level = pd.concat([df_noise_level, temp], axis=0)

# change to loc+'export_42/**/' for reading all months, but the kernel will crash

In [ ]:
# no missing values
# there are some missing values in certain month, but not in all months
print_na_percent(df_noise_level)

In [ ]:
# the number of unique locations for the noise level
len(df_noise_level['description'].unique())

In [5]:
df_noise_agg = pd.read_csv('/Users/wentingjiang/Desktop/mda_kul_local/df_noise_agg.csv')
df_noise_agg.head()

,Unnamed: 0,timestamp,MP 01: Naamsestraat 35 Maxim,MP 02: Naamsestraat 57 Xior,MP 03: Naamsestraat 62 Taste,MP 04: His & Hears,MP 05: Calvariekapel KU Leuven,MP 06: Parkstraat 2 La Filosovia,MP 07: Naamsestraat 81,MP 08: bis - Vrijthof
0,0,2022-01-03 00:00,70.0,50.8,52.0,NaN,66.7,65.5,67.6,NaN
1,1,2022-01-03 00:01,71.8,54.6,64.7,NaN,62.8,61.0,57.3,NaN
2,2,2022-01-03 00:02,72.6,67.7,72.0,NaN,68.4,62.2,65.6,NaN
3,3,2022-01-03 00:03,67.0,64.8,62.2,NaN,54.5,67.3,74.1,NaN
4,4,2022-01-03 00:04,73.3,66.5,62.1,NaN,70.6,78.2,67.9,NaN


In [23]:
#for each timestamp, not all locations have the noise level for that time stamp
df_noise_agg.isna().sum()

description
timestamp                           27360
MP 01: Naamsestraat 35  Maxim           0
MP 02: Naamsestraat 57 Xior             0
MP 03: Naamsestraat 62 Taste            0
MP 05: Calvariekapel KU Leuven          0
MP 06: Parkstraat 2 La Filosovia        0
MP 07: Naamsestraat 81                 66
MP 08: bis - Vrijthof                   0
dtype: int64

In [8]:
df_noise_agg_no_nan = df_noise_agg[~df_noise_agg['timestamp'].isna()]
df_noise_agg_no_nan.shape
df_noise_agg_no_nan.head()

,Unnamed: 0,timestamp,MP 01: Naamsestraat 35 Maxim,MP 02: Naamsestraat 57 Xior,MP 03: Naamsestraat 62 Taste,MP 04: His & Hears,MP 05: Calvariekapel KU Leuven,MP 06: Parkstraat 2 La Filosovia,MP 07: Naamsestraat 81,MP 08: bis - Vrijthof
0,0,2022-01-03 00:00,70.0,50.8,52.0,NaN,66.7,65.5,67.6,NaN
1,1,2022-01-03 00:01,71.8,54.6,64.7,NaN,62.8,61.0,57.3,NaN
2,2,2022-01-03 00:02,72.6,67.7,72.0,NaN,68.4,62.2,65.6,NaN
3,3,2022-01-03 00:03,67.0,64.8,62.2,NaN,54.5,67.3,74.1,NaN
4,4,2022-01-03 00:04,73.3,66.5,62.1,NaN,70.6,78.2,67.9,NaN


## Noise events

In [21]:
df_noise_events = read_noise_data(loc+'export_41/')
df_noise_events2 =truncate_time_to_min(df_noise_events)
df_noise_events_detected=remove_noise_undetected(df_noise_events2)
df_noise_events_detected.shape
df_noise_events_detected.groupby('noise_event_laeq_primary_detected_class').count()
# there are a lot of noise event not categorized in a group

,#object_id,description,noise_event_laeq_model_id,noise_event_laeq_model_id_unit,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_certainty_unit,noise_event_laeq_primary_detected_class_unit,timestamp
noise_event_laeq_primary_detected_class,,,,,,,,
Human voice - Shouting,5742,5742,5742,0,5742,5742,0,2320
Human voice - Singing,1082,1082,1082,0,1082,1082,0,620
Music non-amplified,126,126,126,0,126,126,0,40
Nature elements - Wind,113,113,113,0,113,113,0,48
Transport road - Passenger car,47388,47388,47388,0,47388,47388,0,16704
Transport road - Siren,1992,1992,1992,0,1992,1992,0,805
Unsupported,20791,20791,20791,0,20791,20791,0,8217


In [22]:
df_noise_events_detected['description'].unique()
# only 7 location as well

array(['MP 07: Naamsestraat 81', 'MP 06: Parkstraat 2 La Filosovia',
       'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste',
       'MP 01: Naamsestraat 35  Maxim', 'MP 08: bis - Vrijthof',
       'MP 05: Calvariekapel KU Leuven'], dtype=object)

In [23]:
df_noise_type = df_noise_events_detected.groupby(['description',
                                      'timestamp',
        'noise_event_laeq_primary_detected_class']).count().reset_index()
df_noise_filter = df_noise_type[['description', 'timestamp', 'noise_event_laeq_primary_detected_class',
       '#object_id']]
df_noise_filter.columns =['location', 'timestamp', 'event', 'count']
#df_noise_filter.to_csv('noise_events_count.csv')

## Join noise level and events data

In [25]:
noise_level_long_df = pd.melt(df_noise_agg_no_nan, id_vars='timestamp', value_vars=[
    'MP 01: Naamsestraat 35  Maxim',
    'MP 02: Naamsestraat 57 Xior',
    'MP 03: Naamsestraat 62 Taste',
   # 'MP 04: His & Hears',
    'MP 05: Calvariekapel KU Leuven',
    'MP 06: Parkstraat 2 La Filosovia',
    'MP 07: Naamsestraat 81',
    'MP 08: bis - Vrijthof'
], var_name='location', value_name='noise_level')
noise_level_long_df.head()

,timestamp,location,noise_level
0,2022-01-03 00:00,MP 01: Naamsestraat 35 Maxim,70.0
1,2022-01-03 00:01,MP 01: Naamsestraat 35 Maxim,71.8
2,2022-01-03 00:02,MP 01: Naamsestraat 35 Maxim,72.6
3,2022-01-03 00:03,MP 01: Naamsestraat 35 Maxim,67.0
4,2022-01-03 00:04,MP 01: Naamsestraat 35 Maxim,73.3


In [26]:
# merge noise events type to the minute level noise level
merged_df = noise_level_long_df.merge(df_noise_filter, on=['location','timestamp'], how='left')

In [28]:
merged_df['count'] = merged_df['count'].fillna(0)
merged_df['event'] = merged_df['event'].fillna('None')
# it could be that at a certain minute, there are multiple events, ignore this for now
merged_df.head()

,timestamp,location,noise_level,event,count
0,2022-01-03 00:00,MP 01: Naamsestraat 35 Maxim,70.0,None,0.0
1,2022-01-03 00:01,MP 01: Naamsestraat 35 Maxim,71.8,None,0.0
2,2022-01-03 00:02,MP 01: Naamsestraat 35 Maxim,72.6,None,0.0
3,2022-01-03 00:03,MP 01: Naamsestraat 35 Maxim,67.0,None,0.0
4,2022-01-03 00:04,MP 01: Naamsestraat 35 Maxim,73.3,None,0.0


In [29]:
merged_df.head()

test2 = pd.pivot_table(merged_df, values='count', index=['timestamp', 'location'], columns='event')
test3 = test2.reset_index()
test3.shape

(1088640, 10)

In [30]:
test=merged_df.groupby(['location', 'timestamp']).count().reset_index()
test[test['event']>1]

# need to remove duplicate, by location and timestamp (just take the first event)
# not a significant portion anyway

,location,timestamp,noise_level,event,count
2096,MP 01: Naamsestraat 35 Maxim,2022-01-04 10:56,2,2,2
6377,MP 01: Naamsestraat 35 Maxim,2022-01-07 10:17,2,2,2
8701,MP 01: Naamsestraat 35 Maxim,2022-01-10 01:01,2,2,2
9800,MP 01: Naamsestraat 35 Maxim,2022-01-10 19:20,2,2,2
11543,MP 01: Naamsestraat 35 Maxim,2022-01-12 00:23,2,2,2
...,...,...,...,...,...
1057780,MP 08: bis - Vrijthof,2022-10-08 13:40,2,2,2
1057781,MP 08: bis - Vrijthof,2022-10-08 13:41,2,2,2
1057782,MP 08: bis - Vrijthof,2022-10-08 13:42,2,2,2
1057951,MP 08: bis - Vrijthof,2022-10-08 16:31,2,2,2


In [31]:
merged_df['event'].unique() # there different types of event
# but on a minute basis, most of the time, there are no events

array(['None', 'Human voice - Shouting', 'Transport road - Siren',
       'Unsupported', 'Transport road - Passenger car',
       'Human voice - Singing', 'Nature elements - Wind',
       'Music non-amplified'], dtype=object)

In [32]:
merged_df_ready = merged_df.copy()

# Add datetime features

In [33]:
# Convert the timestamp column to a datetime object
merged_df_ready['timestamp'] = pd.to_datetime(merged_df_ready['timestamp'])

# Create new columns for hour and day of the week
merged_df_ready['hour'] = merged_df_ready['timestamp'].dt.hour
merged_df_ready['day_of_week'] = merged_df_ready['timestamp'].dt.day_name()

# only include the time after 18:00, as Leuven is interested in understanding noise at night
merged_df_ready=merged_df_ready[(merged_df_ready['hour']>=17) |
                                     (merged_df_ready['hour']<=3)]
#merged_df_ready=pd.get_dummies(merged_df_ready_small)

# Add weather feature more precisely

In [36]:
# the relevant weather station that is within 200 meters from the noise location
# this dict is extracted by notebook: map_weather_stations_to_noise_location
weather_noise_location_map={'Naamsestraat 35  Maxim': ['LC-102', 'LC-117'],
 'Naamsestraat 62 Taste': ['LC-112', 'LC-117', 'LC-118'],
 'Calvariekapel KU Leuven': ['LC-112', 'LC-118'],
 'Parkstraat 2 La Filosovia': ['LC-109', 'LC-112', 'LC-118'],
 'Naamsestraat 57 Xior': ['LC-117'],
 'Naamsestraat 81': ['LC-109', 'LC-112', 'LC-118'],
 #'His & Hears': ['LC-112', 'LC-118']
                           }

In [ ]:
folder='/data'
all_file = ['Q1', 'Q2', 'Q3', 'Q4']

df_weather = pd.DataFrame()
for file in all_file:
    temp=pd.read_csv(folder+'LC_2022'+ file+'.csv') # read all quarters
    df_weather=pd.concat([df_weather, temp], axis=0)

df_meta= pd.read_csv(folder+'01_Metadata_v2.csv')
df = df_weather.merge(df_meta, on='ID', how='left')

In [40]:
temp=pd.DataFrame()
for key, value in weather_noise_location_map.items():
    df_filtered = df[df['ID'].isin(value)]
    df_filtered['location'] = key
    temp=pd.concat([temp, df_filtered], axis=0)
temp.head()

/var/folders/ps/310fvxcx6y55_xvdkg77780h0000gn/T/ipykernel_72169/222578823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['location'] = key
/var/folders/ps/310fvxcx6y55_xvdkg77780h0000gn/T/ipykernel_72169/222578823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['location'] = key
/var/folders/ps/310fvxcx6y55_xvdkg77780h0000gn/T/ipykernel_72169/222578823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,DATEUTC,ID,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,...,LC_TEMP_QCL3,LAT,LON,ALT,LCZ,LC,building_height,SVF,installation_height,location
946080,2022-01-01 00:10:00,LC-102,90.0,11.95,38.0,0.0,0.0,0.0,0.0,0.0,...,13.23452,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946081,2022-01-01 00:20:00,LC-102,89.0,11.78,37.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946082,2022-01-01 00:30:00,LC-102,89.0,11.78,38.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946083,2022-01-01 00:40:00,LC-102,89.0,11.78,37.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946084,2022-01-01 00:50:00,LC-102,89.0,11.78,38.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim


In [41]:
temp['location'].unique() #  6 locations?

array(['Naamsestraat 35  Maxim', 'Naamsestraat 62 Taste',
       'Calvariekapel KU Leuven', 'Parkstraat 2 La Filosovia',
       'Naamsestraat 57 Xior', 'Naamsestraat 81'], dtype=object)

In [42]:
feature_names_simplified = {'DATEUTC': 'timestamp_10min',
                            'LC_HUMIDITY': 'humidity', 'LC_DWPTEMP': 'temp', 'LC_RAD': 'solar_radiation',
                'LC_RAININ': 'rain_intensity', 'LC_DAILYRAIN': 'daily_rain_sum', 'LC_WINDDIR': 'wind_dir',
                 'LC_WINDSPEED': 'wind_speed', 'LAT': 'lat', 'LON': 'lon', 'ALT': 'alt', 'LCZ': 'climate_zone',
                 'LC': 'green_cover', 'building_height': 'building_height', 'SVF': 'sky_view_factor',
                 'installation_height': 'installation_height'
                }

temp = temp.rename(columns=feature_names_simplified)
temp.head()

,timestamp_10min,ID,humidity,temp,LC_n,solar_radiation,rain_intensity,daily_rain_sum,wind_dir,wind_speed,...,LC_TEMP_QCL3,lat,lon,alt,climate_zone,green_cover,building_height,sky_view_factor,installation_height,location
946080,2022-01-01 00:10:00,LC-102,90.0,11.95,38.0,0.0,0.0,0.0,0.0,0.0,...,13.23452,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946081,2022-01-01 00:20:00,LC-102,89.0,11.78,37.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946082,2022-01-01 00:30:00,LC-102,89.0,11.78,38.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946083,2022-01-01 00:40:00,LC-102,89.0,11.78,37.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim
946084,2022-01-01 00:50:00,LC-102,89.0,11.78,38.0,0.0,0.0,0.0,0.0,0.0,...,13.20297,50.878,4.701,27,2,0,16.051847,0.647079,3,Naamsestraat 35 Maxim


In [49]:
# decide what features to include

features =['humidity', 'temp', #'LC_n', 
           'solar_radiation',
       'rain_intensity', 'daily_rain_sum', 'wind_dir', 'wind_speed', 
         
         #  'Date', 'Year', 'Month', 'Day', 'Hour', 'Minute', 
           'LC_RAD60', 
           'LC_TEMP_QCL0', 'LC_TEMP_QCL1', 'LC_TEMP_QCL2', 'LC_TEMP_QCL3', 
           'lat', 'lon', 
           'alt',
       'climate_zone', 'green_cover', 'building_height', 'sky_view_factor',
       'installation_height']

weather_sample_renamed = temp.groupby(['location', 
             'timestamp_10min'])[features].median().reset_index()

weather_sample_renamed.head() # now merge this by location and time to the noise level data
# need to rename the location

,location,timestamp_10min,humidity,temp,solar_radiation,rain_intensity,daily_rain_sum,wind_dir,wind_speed,LC_RAD60,...,LC_TEMP_QCL2,LC_TEMP_QCL3,lat,lon,alt,climate_zone,green_cover,building_height,sky_view_factor,installation_height
0,Calvariekapel KU Leuven,2022-01-01 00:10:00,91.0,11.715,0.0,0.0,0.0,151.5,0.075,0.0,...,13.07725,13.107725,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
1,Calvariekapel KU Leuven,2022-01-01 00:20:00,90.5,11.640,0.0,0.0,0.0,57.0,0.145,0.0,...,13.07725,13.135080,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
2,Calvariekapel KU Leuven,2022-01-01 00:30:00,90.5,11.640,0.0,0.0,0.0,56.0,0.085,0.0,...,13.07725,13.071885,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
3,Calvariekapel KU Leuven,2022-01-01 00:40:00,90.5,11.640,0.0,0.0,0.0,37.0,0.040,0.0,...,13.07725,13.059835,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
4,Calvariekapel KU Leuven,2022-01-01 00:50:00,90.5,11.635,0.0,0.0,0.0,29.0,0.050,0.0,...,13.07725,13.066240,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0


In [50]:
weather_sample_renamed['location'].unique()

array(['Calvariekapel KU Leuven', 'Naamsestraat 35  Maxim',
       'Naamsestraat 57 Xior', 'Naamsestraat 62 Taste', 'Naamsestraat 81',
       'Parkstraat 2 La Filosovia'], dtype=object)

In [51]:
merged_df_ready['location'].unique()

array(['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior',
       'MP 03: Naamsestraat 62 Taste', 'MP 05: Calvariekapel KU Leuven',
       'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81',
       'MP 08: bis - Vrijthof'], dtype=object)

In [52]:
mapping_dict = {'Calvariekapel KU Leuven': 'MP 05: Calvariekapel KU Leuven',
               'Naamsestraat 35  Maxim': 'MP 01: Naamsestraat 35  Maxim',
               'Naamsestraat 57 Xior': 'MP 02: Naamsestraat 57 Xior',
                'Naamsestraat 62 Taste': 'MP 03: Naamsestraat 62 Taste',
                'Naamsestraat 81': 'MP 07: Naamsestraat 81',
                'Parkstraat 2 La Filosovia': 'MP 06: Parkstraat 2 La Filosovia'
               }

In [53]:
weather_sample_renamed['location'] =weather_sample_renamed['location'].replace(mapping_dict)
weather_sample_renamed.head()

,location,timestamp_10min,humidity,temp,solar_radiation,rain_intensity,daily_rain_sum,wind_dir,wind_speed,LC_RAD60,...,LC_TEMP_QCL2,LC_TEMP_QCL3,lat,lon,alt,climate_zone,green_cover,building_height,sky_view_factor,installation_height
0,MP 05: Calvariekapel KU Leuven,2022-01-01 00:10:00,91.0,11.715,0.0,0.0,0.0,151.5,0.075,0.0,...,13.07725,13.107725,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
1,MP 05: Calvariekapel KU Leuven,2022-01-01 00:20:00,90.5,11.640,0.0,0.0,0.0,57.0,0.145,0.0,...,13.07725,13.135080,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
2,MP 05: Calvariekapel KU Leuven,2022-01-01 00:30:00,90.5,11.640,0.0,0.0,0.0,56.0,0.085,0.0,...,13.07725,13.071885,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
3,MP 05: Calvariekapel KU Leuven,2022-01-01 00:40:00,90.5,11.640,0.0,0.0,0.0,37.0,0.040,0.0,...,13.07725,13.059835,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0
4,MP 05: Calvariekapel KU Leuven,2022-01-01 00:50:00,90.5,11.635,0.0,0.0,0.0,29.0,0.050,0.0,...,13.07725,13.066240,50.875,4.7,40.0,2.0,0.0,17.767563,0.674119,3.0


# Merge noise and weather data

In [ ]:
df = merged_df_ready.copy()
df2 = df[~df['timestamp'].isna()]
df2['timestamp_round'] = df2['timestamp'].dt.floor('10min').dt.strftime('%Y-%m-%d %H:%M')


In [57]:
weather_sample_renamed['timestamp_10min'] = pd.to_datetime(weather_sample_renamed['timestamp_10min'])
df2['timestamp_round'] = pd.to_datetime(df2['timestamp_round'])

In [ ]:
merged_df_final = pd.merge(df2, weather_sample_renamed, left_on=['timestamp_round', 'location'], right_on=['timestamp_10min', 'location'], how='left')

In [69]:
df_clean = merged_df_final[~merged_df_final['timestamp_10min'].isna()]

df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])  # Convert the 'timestamp' column to datetime if needed

df_clean['month'] = df_clean['timestamp'].dt.month
df_clean.head()

time_col_to_drop = ['timestamp_round', 'timestamp_10min', #'timestamp',
                #   'Date', 'Year', 'Month', 'Day', 'Hour'
                   ]
df_model = df_clean.drop(time_col_to_drop, axis=1)#.drop('location', axis=1)

df_model_no_nan = df_model[~df_model['noise_level'].isna()]
df_model_no_nan = df_model_no_nan[~df_model_no_nan['LC_TEMP_QCL1'].isna()]

df_model_no_nan.isna().sum()

timestamp              0
location               0
noise_level            0
event                  0
count                  0
hour                   0
day_of_week            0
humidity               0
temp                   0
solar_radiation        0
rain_intensity         0
daily_rain_sum         0
wind_dir               0
wind_speed             0
LC_RAD60               0
LC_TEMP_QCL0           0
LC_TEMP_QCL1           0
LC_TEMP_QCL2           0
LC_TEMP_QCL3           0
lat                    0
lon                    0
alt                    0
climate_zone           0
green_cover            0
building_height        0
sky_view_factor        0
installation_height    0
month                  0
dtype: int64

# Model

In [74]:
df_model = df_model_no_nan.copy()
df_model['day_of_month'] = df_model['timestamp'].dt.day

subset_cols = ['location', 'month', 'day_of_month', 'hour']
# Remove duplicates based on the subset of columns
df_model_hourly = df_model.drop_duplicates(subset=subset_cols)